See description in [BestAddressAnomalies.md](BestAddressAnomalies.md)

# Main code

In [ ]:
import os
import urllib

import geopandas as gpd
import pandas as pd
import numpy as np

import pickle
import contextily as ctx
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

from tqdm import tqdm, trange
tqdm.pandas()

import re
import jellyfish

import difflib

In [ ]:
import shapely

from shapely.geometry import Point, LineString


import warnings
warnings.filterwarnings("ignore", category=shapely.errors.ShapelyDeprecationWarning) 

## Functions

In [ ]:
def download_if_nexist(url, filename):
    """
    If the (local) file <filename> does not exists, download it from <url>

    Parameters
    ----------
    url: str
       url to fetch
    filename: str
       local file to save

    Returns
    -------

    None
    """
    if not os.path.isfile(filename):
        #gcontext = ssl.SSLContext()
        with urllib.request.urlopen(url) as response:
            with open(filename, "wb") as f:
                f.write(response.read())

In [ ]:
def set_optimal_limits(ax, df):
    """
    Adapt xlim/ylim to a GeoDataFrame point plot to avoid plot to be too wide when 
    points are horizontally aligned, and to narrow when points are vertically aligned

    Usage : 
    
    ax = df.plot()
    set_optimal_limits(ax, df)
    
    Parameters
    ----------
    ax: AxesSubplot
       plot to resize
    df: GeoDataFrame
       data to be plotted

    Returns
    -------
        None
    """
    
    plot_ratio = 1.5 # optimal ratio between "one horizontal degree" and "one vertical degree". It depends of the CRS. 
                     # For "polar" CRS, it may also depend of the place on the globe

    minimal_width=400
   
    margins = 1.1 # Avoid having dots on edges of the plot
    

    # Compute dimension of the data
    xmin, ymin, xmax, ymax = df.total_bounds
    height = (ymax - ymin) 
    width = (xmax - xmin)
    
    opt_height = max(height, width / plot_ratio, minimal_width / plot_ratio)
    opt_width  = max(width , height*plot_ratio, minimal_width)
    
#     print(xmin, ymin, xmax, ymax)
#     print(width, height, opt_width, opt_height)
    # If plot is too narrow, increase xmin. If plot is too wide, increase ylim

    if opt_height > height :
        ymid = (ymax+ymin)/2
        mid_height = opt_height * margins / 2
        ax.set_ylim(ymid - mid_height, ymid + mid_height)
    if opt_width > width:
        xmid = (xmax+xmin)/2
        mid_width = opt_width* margins/2
        ax.set_xlim(xmid - mid_width, xmid + mid_width)

In [ ]:
def null_jaro(str1, str2):
    if pd.isnull(str1) or pd.isnull(str2):
        return pd.NA
    
    return jellyfish.jaro_winkler_similarity(str1, str2)

In [ ]:

def line_sinuosity(geom):
    # Return the ratio between the distance of the geometry, and the straight distance between the start and the end of the geometry
    assert geom.geom_type == "LineString", geom.geom_type
    length = geom.length
    start_pt = geom.interpolate(0)
    end_pt = geom.interpolate(1, normalized=True)
    straight_dist = start_pt.distance(end_pt)
    if straight_dist == 0.0:
        if length == 0.0:
            return 0.0
        return float("inf")
    return length / straight_dist

In [ ]:
def sliding_sinuosity(street_side, windows_size=5):
    if street_side.geometry.nunique() <3:
        return pd.NA
    geometry = street_side.geometry
    local_sinuosity = [line_sinuosity(LineString(geometry.iloc[i:i+windows_size].reset_index(drop=True))) for i in range(0, max(1, geometry.shape[0]-windows_size))]
#     print(local_sinuosity)
    
    return np.mean(local_sinuosity)
                                                                                                          
                                      
    
    

In [ ]:
# region[region.streetname == "Abbaye de la Cambre"].geometry.iloc[5:10].reset_index(drop=True)

In [ ]:
# region[(region.house_number_num.mod(2)==parity)].groupby(["streetname", "postcode"]).geometry.progress_apply(sliding_sinuosity)


In [ ]:
def bloc_sinuosity(street_side):
    if street_side.geometry.nunique() <3:
        return pd.NA
    street_side_linestring = LineString(street_side.reset_index().geometry)
    
    return line_sinuosity(street_side_linestring)

In [ ]:
def bloc_length(street_side):
    if street_side.geometry.nunique() <3:
        return pd.NA
    street_side_linestring = LineString(street_side.reset_index().geometry)
    
    return street_side_linestring.length

In [ ]:
# bloc_length(region[region.streetname == "Abbaye de la Cambre"])

In [ ]:
def get_max_delta_ratio(street_bloc):
    if  street_bloc.house_number_num.max() - street_bloc.house_number_num.min() < 20:
        return 0, "-"
    
    b1 = street_bloc.reset_index()[["house_number_num", "geometry", "index"]].assign(mg=1)
    b2 = street_bloc.reset_index()[["house_number_num", "geometry", "index"]].assign(mg=1)
    dot_prod = b1.merge(b2, on="mg")
    dot_prod = dot_prod[dot_prod.index_x < dot_prod.index_y]
    
    dot_prod["delta_num"] = (dot_prod.house_number_num_x - dot_prod.house_number_num_y).abs()
    
    dot_prod = dot_prod[dot_prod["delta_num"]>10]
    
    dot_prod["distance"] = gpd.GeoSeries(dot_prod.geometry_x).distance(gpd.GeoSeries(dot_prod.geometry_y))
    dot_prod["delta_ratio"] = dot_prod.delta_num/dot_prod.distance
    
    dot_prod = dot_prod[dot_prod.delta_ratio<np.inf]
    
    if dot_prod.shape[0]==0:
        return 0, "-"
    
    id_max = dot_prod.delta_ratio.idxmax()
    
    rec_max = dot_prod.loc[id_max]
    return (rec_max.delta_ratio, f"{rec_max.house_number_num_x} -> {rec_max.house_number_num_y}") 
#     display(dot_prod.sort_values("delta_ratio"))
#     return  .delta_ratio.max()
#     return f"{dot_prod.loc[imax].delta_ratio} ({})


In [ ]:
def get_street_bloc(region, streetname, postcode, parity):
    street_bloc=region[(region.streetname==streetname) & (region.postcode==postcode) & (region.house_number_num.mod(2)==parity) ].copy()
    
    street_bloc = street_bloc.sort_values(["house_number_num", "house_number"])

    street_bloc = street_bloc.drop_duplicates(["geometry", "house_number"])
    
    return street_bloc

In [ ]:
def plot_street_bloc(street_bloc, title=None, ax=None):
    
    ax = street_bloc.plot(column="house_number_num", figsize=(10,10),alpha=0.7, ax=ax, zorder=10)
    
    ax.plot(street_bloc.geometry.x.values, street_bloc.geometry.y.values, alpha=0.5)#, ax=ax)#, ax=ax, kind="line")
#     ax.title = title
    ax.set_title(title)

    for idx, row in street_bloc.iterrows():
    #     print(row["housenumber"])
        an = ax.annotate(text=row["house_number"], 
                         xy=(row["geometry"].x, row["geometry"].y), 
                         fontsize="x-small",
                         xytext=(5, 5), textcoords='offset pixels',)
        
#     ax = street_bloc.plot(column="house_number_num", alpha=1, ax=ax)
    set_optimal_limits(ax, street_bloc)

    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)


In [ ]:
# def plot_street_bloc(street_bloc, title=None, ax=None):
    
# #     if ax:
# #         ax=ax.plot(street_bloc.geometry.x.values, street_bloc.geometry.y.values, alpha=0.5)
# #     else:
# #         ax=plt.plot(street_bloc.geometry.x.values, street_bloc.geometry.y.values, alpha=0.5)[0]#, ax=ax, kind="line")
# #     print(ax)
# #     ax.title = title
#     ax = street_bloc.plot(street_bloc.geometry.x.values, street_bloc.geometry.y.values, alpha=0.5)

#     for idx, row in street_bloc.iterrows():
#     #     print(row["housenumber"])
#         an = ax.annotate(text=row["house_number"], xy=(row["geometry"].x, row["geometry"].y), fontsize="x-small")
#         an.set_zorder(10)
#     set_optimal_limits(ax, street_bloc)

#     ax = street_bloc.plot(column="house_number_num", alpha=0.5, ax=ax, figsize=(10,10))
#     ax.set_title(title)

#     ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)


In [ ]:
# plot_street_bloc(street_bloc, "title")

In [ ]:
import plotly.graph_objects as go

def plot_street_bloc_plotly(street_bloc):
    street_bloc_osm = street_bloc.to_crs(osm_crs)


    fig = go.Figure(go.Scattermapbox(
        mode = "markers+lines",
    #     width=950, height=800,
        lat = street_bloc_osm.geometry.y,
        lon = street_bloc_osm.geometry.x,

    #     color= ,
        marker=go.scattermapbox.Marker(
                    #line=dict(width = 1),
                    color = street_bloc_osm['house_number_num'],
                    size = 10
                ),

            hovertext=street_bloc_osm[["house_number", "streetname", "postcode", "postname", "municipality"]]
        ))


    fig.update_layout(
    #    margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
            'center': {'lon': street_bloc_osm.geometry.x.median(), 
                       'lat': street_bloc_osm.geometry.y.median()},
            'style': "open-street-map",

    #         'center': {'lon': -20, 'lat': -20},
             'zoom': 10,

        },
    width=950, height=800,
    )


    fig.show()
    return fig
    


In [ ]:
def make_table(street_bloc, title=None):
    street_bloc = street_bloc.reset_index()
    
    street_bloc["dist_to_prev"] = street_bloc.distance(street_bloc.shift(1)).fillna(0).astype(int)
    
    if street_bloc.shape[0]>20:
        df1 = street_bloc[["house_number", "address_id", "geometry", "dist_to_prev"]].head(10)
        df2 = street_bloc[["house_number", "address_id", "geometry", "dist_to_prev"]].tail(10)
        df3=pd.DataFrame(columns=df1.columns, 
                         index=["..."],
                         data = [ ["..."]*df1.shape[1]])

        df = pd.concat([df1, df3, df2])
    else:
        df = street_bloc[["house_number",  "address_id", "geometry", "dist_to_prev"]].copy()
    
    df["address_id"] = df["address_id"].apply(";".join)
    for fld in ["address_id"]:
        df[fld]  = df[fld].str[0:30]

    fig, ax = plt.subplots(figsize=(10,.3*df.shape[0]))
    ax.axis('tight')
    ax.axis('off')

    the_table = ax.table(cellText=df.values,
                            rowLabels=df.index,
                            colLabels=df.columns,
                            rowColours=['lightblue']*len(df),
                            colColours=['lightblue']*len(df.columns),
    #                         cellColours=alternating_colors,
                            loc='center')
    plt.title(title)
    return fig


In [ ]:
def make_metric_table(rec, metrics, ax=None):
    metrics_table = pd.DataFrame({m: {
        "metric": f"{rec[m]:.2f}",
        "house_number": rec[f"{m}_house_number"] if f"{m}_house_number" in rec else "",
        "ranking" : "" if pd.isnull(rec[f"{m}_ranking"]) else int(rec[f"{m}_ranking"]) ,
    } for m in metrics})

    if ax is None:
        fig, ax = plt.subplots(figsize=(10,1))
    ax.axis('tight')
    ax.axis('off')

    the_table = ax.table(cellText=metrics_table.values,
                            rowLabels=metrics_table.index,
                            colLabels=metrics_table.columns,
                            rowColours=['lightblue']*len(metrics_table),
                            colColours=['lightblue']*len(metrics_table.columns),
    #                         cellColours=alternating_colors,
                            loc='center')
    #return fig

In [ ]:
def plot_building_boxes(boxes, full_region, with_explore=False):
    #rec=region_buildings.iloc[[k]]
    r= boxes.reset_index().iloc[0]
    #title = f"{r.streetname}, {r.house_number}, {r.postcode} {r.municipality} ({int(r.ch_perimeter)} m, {int(r.ch_area/10000)} ha)"
    title = f"{r.streetname}, {r.house_number}, {r.postcode}/{r.municipality_id} {r.municipality} ({int(r.mrr_length)} m x {int(r.mrr_width)} m)"

    if with_explore:
        m=boxes.explore()
        return boxes.set_geometry("min_rot_rect").explore(m=m,  color="red")
    else:
        ax = boxes.plot()
        boxes.set_geometry("min_rot_rect").plot(ax=ax, alpha=0.2, color="red")
        ax.set_title(title)

        set_optimal_limits(ax, boxes)
        ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

        all_boxes = boxes.reset_index()[boxes.index.names].merge(full_region)
    #     display(all_boxes)
        for idx, row in all_boxes.fillna("/").iterrows():
        #     print(row["housenumber"])
            ax.annotate(text=row["box_number"], xy=(row["geometry"].x, row["geometry"].y))

    

In [ ]:
def get_min_rot_rect_size(min_rot_rect):
    if isinstance(min_rot_rect, Point):
        return 0,0
    
    if isinstance(min_rot_rect, LineString):
        return min_rot_rect.length, 0
    x, y = min_rot_rect.exterior.coords.xy

    # get length of bounding box edges
    edge_length = (Point(x[0], y[0]).distance(Point(x[1], y[1])), Point(x[1], y[1]).distance(Point(x[2], y[2])))

    # get length of polygon as the longest edge of the bounding box
    length = max(edge_length)

    # get width of polygon as the shortest edge of the bounding box
    width = min(edge_length)

    return length, width

## Parameters

In [ ]:
root_output_dir = "output/best_anomalies"

data_dir = "data/best_anomalies"

topn = 50

In [ ]:
# os.makedirs(output_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# datadir = "data/geocoding/"

In [ ]:
region_name = "bru"
# region_name = "vlg"
# region_name = "wal"

In [ ]:
municipality_id_prefix = None
# municipality_id_prefix = "25"
# municipality_id_prefix = "25112" # Wavre
# municipality_id_prefix = "93088" # Walcourt

In [ ]:
if municipality_id_prefix is not None:
    region_name = f"{region_name}_{municipality_id_prefix}"

In [ ]:
output_dir = f"{root_output_dir}/{region_name}"
os.makedirs(output_dir, exist_ok=True)

## Data reading

In [ ]:
best_fn = f"{data_dir}/openaddress-be{region_name}.zip"
download_if_nexist(f"https://opendata.bosa.be/download/best/openaddress-be{region_name}.zip", best_fn)


In [ ]:
full_region = pd.read_csv(best_fn, usecols=["municipality_name_de", "municipality_name_fr", "municipality_name_nl", "municipality_id",
                                            "streetname_de", "streetname_fr", "streetname_nl", "street_id",
                                            "postname_fr", "postname_nl",
                                            "postcode", "house_number", "box_number", "region_code", 
                                            "EPSG:31370_x", "EPSG:31370_y", 
                                            "EPSG:4326_lat", "EPSG:4326_lon", 
                                            "address_id", "status"], dtype=str)
full_region

In [ ]:
# full_region[full_region.streetname_fr=="Rue François Michoel"].iloc[0:60]

In [ ]:
if municipality_id_prefix is not None:
    full_region = full_region[full_region.municipality_id.str.startswith(municipality_id_prefix)].copy()

In [ ]:
# region[(region.streetname_fr=="Rue de Wérister") & (region.house_number=="7")]

In [ ]:
print("Without coordinates : ")
full_region[full_region["EPSG:31370_x"]== "0.00000"].sort_values(["postcode", "streetname_fr"])#.iloc[0:60]

In [ ]:
full_region = full_region[full_region["EPSG:31370_x"]!= "0.00000"].copy()

In [ ]:
full_region.status.value_counts()#/ region.shape[0]

In [ ]:
full_region = full_region[full_region.status=="current"]

In [ ]:
full_region["streetname"] =   full_region.streetname_fr.fillna(full_region.streetname_nl).fillna(full_region.streetname_de)
full_region["municipality"] = full_region.municipality_name_fr.fillna(full_region.municipality_name_nl).fillna(full_region.municipality_name_de)
full_region["postname"] =     full_region.postname_fr.fillna(full_region.postname_nl).fillna("[na]")

full_region["house_number_num"]= full_region.house_number.str.extract("^([0-9]*)").astype(int, errors="ignore")


In [ ]:
print("No numerical house number:")
print(full_region[full_region.house_number_num== ""].shape[0])
full_region[full_region.house_number_num== ""]

In [ ]:
full_region = full_region[full_region.house_number_num!= ""]
full_region.house_number_num = full_region.house_number_num.astype(int)

In [ ]:
# region

In [ ]:
crs = "epsg:3857"
full_region["geometry"] = gpd.points_from_xy(full_region["EPSG:31370_x"], full_region["EPSG:31370_y"])
full_region = gpd.GeoDataFrame(full_region)
full_region = full_region.set_crs("epsg:31370").to_crs(crs)
full_region = full_region.drop(["EPSG:31370_x", "EPSG:31370_y"], axis=1)

In [ ]:
#Workaround as geometries are not "groupbyable"(/hashable)
full_region_wkb = full_region.assign(geometry_wkb= full_region.geometry.apply(lambda geom: geom.wkb))
                                     
region = full_region_wkb.fillna("[na]").groupby(["streetname", "house_number", "house_number_num", "postcode", "postname", "municipality", "municipality_id", "geometry_wkb"]).address_id.progress_apply(list).reset_index()
region = region.merge(full_region_wkb[["geometry_wkb", "geometry"]].drop_duplicates()).drop("geometry_wkb", axis=1)

region= gpd.GeoDataFrame(region)
del full_region_wkb

In [ ]:
region = region.sort_values(["postcode", "streetname", "house_number_num"])

In [ ]:
region

# Box anomalies

In [ ]:
# Several names for the same street id
x = full_region[["street_id", "streetname"]].drop_duplicates()
x[x.street_id.duplicated()]

In [ ]:
# Street_ids in multiple municipalities
x = full_region[["street_id", "municipality_id"]].drop_duplicates()
x[x.street_id.duplicated(keep=False)].sort_values("street_id").merge(full_region[["street_id", "municipality_id", "streetname", "municipality"]].drop_duplicates())

In [ ]:
# Several ids for the same street
x = full_region[["street_id", "municipality_id", "streetname", "postcode"]].drop_duplicates()
x[x.duplicated(subset=["streetname", "municipality_id", "postcode"], keep=False)].sort_values("streetname")

In [ ]:
# Several streets with the same name (and difference street ids) with the same municipality
x = full_region[["street_id", "municipality_id", "streetname"]].drop_duplicates()
x[x.duplicated(subset=["streetname", "municipality_id"], keep=False)].sort_values("streetname")

In [ ]:
# full_region.

In [ ]:
# Several postcodes for the same building
x = full_region[["street_id", "municipality_id", "house_number", "postcode"]].drop_duplicates()
x[x.duplicated(subset = ["street_id", "municipality_id", "house_number"], keep=False)].sort_values("street_id").merge(full_region[["street_id", "municipality_id","postcode", "streetname", "municipality", "house_number", "box_number", "address_id", "geometry"]].drop_duplicates())

In [ ]:
region_with_boxes = full_region[full_region.duplicated(subset=["municipality_id",  "street_id", "house_number"], keep=False)]

In [ ]:
region_with_boxes

In [ ]:
region_buildings = region_with_boxes[["street_id", "streetname", "municipality_id","municipality", "postcode", "house_number", "geometry"]].dissolve(["street_id", "municipality_id", "postcode", "house_number"])
region_buildings

In [ ]:
region_buildings["convex_hull"] = region_buildings.convex_hull
region_buildings

In [ ]:
region_buildings["min_rot_rect"] = region_buildings.convex_hull.apply(lambda g: g.minimum_rotated_rectangle)
region_buildings

In [ ]:
mrr_size = region_buildings.min_rot_rect.progress_apply(get_min_rot_rect_size)
region_buildings[["mrr_length", "mrr_width"]]=pd.concat([mrr_size.rename("mrr_length").str[0], mrr_size.rename("mrr_width").str[1]], axis=1)

In [ ]:
# region_buildings["ch_perimeter"] = region_buildings.convex_hull.length
# region_buildings["ch_area"] = region_buildings.convex_hull.area

In [ ]:
region_buildings = region_buildings.sort_values("mrr_length", ascending=False)
region_buildings

In [ ]:
# region_buildings.iloc[25:35]

In [ ]:
k=0
plot_building_boxes(region_buildings.iloc[[k]], full_region, with_explore=True)

In [ ]:
pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_box_anomalies.pdf")

region_buildings_sel = region_buildings[(region_buildings.mrr_length > 100) & (region_buildings.mrr_width < 100)]
for k in trange(0, min(2*topn, region_buildings_sel.shape[0])):
    boxes= region_buildings_sel.iloc[[k]]
    
    plot_building_boxes(boxes, full_region)
        
    pdf.savefig(bbox_inches='tight')#, dpi=75)
    plt.close()
    
pdf.close()

In [ ]:
# region_buildings_sel


In [ ]:
del full_region

# Conflict with bPost

In [ ]:
zipcode_boundaries_filename = "data/zipcode_boundaries_shapefile_3812.zip"
download_if_nexist("https://bgu.bpost.be/assets/9738c7c0-5255-11ea-8895-34e12d0f0423_x-shapefile_3812.zip",
                  zipcode_boundaries_filename)

In [ ]:
zipcodes_boundaries = gpd.read_file(f"zip://{zipcode_boundaries_filename}/3812")
zipcodes_boundaries["is_special"] = zipcodes_boundaries.CP_speciau ==1
zipcodes_boundaries = zipcodes_boundaries.rename({"nouveau_PO":"zipcode"}, axis=1)[["zipcode", "is_special", "geometry"]]

zipcodes_boundaries = zipcodes_boundaries.dissolve(["zipcode", "is_special"]).reset_index()

zipcodes_boundaries = zipcodes_boundaries.to_crs(crs)
zipcodes_boundaries

In [ ]:
zipcodes_boundaries["buffer"] = zipcodes_boundaries.buffer(-50)

In [ ]:
region_zipcode = gpd.sjoin(region, zipcodes_boundaries.set_geometry("buffer"))

In [ ]:
region_zipcode.shape, region.shape

In [ ]:
zip_mismatches = region_zipcode[region_zipcode.postcode !=region_zipcode.zipcode]
zip_mismatches

In [ ]:
zip_mismatches.drop(["geometry_left", "geometry_right", "index_right"], axis=1).to_excel(f"{output_dir}/best_anomalies_{region_name}_zip_mismatches.xlsx")

In [ ]:
# mismatches.set_geometry("geometry_left").plot()

In [ ]:
# mismatches.postcode.value_counts().iloc[0:60]

In [ ]:
zip_mismatches.postcode.value_counts()

In [ ]:
# zipcode="1010"
# # display(mismatches[mismatches.postcode==zipcode])
# mism=zip_mismatches[zip_mismatches.postcode==zipcode].set_geometry("geometry_left")
# ax=mism.plot("zipcode", figsize=(15,15),alpha=0.8, legend=True)
# # ax=region[region.postcode==zipcode].plot(figsize=(15,15),alpha=0.8, color="green")
# # ax=mismatches[mismatches.postcode==zipcode].set_geometry("geometry_left").plot(ax=ax, color="red")

# ax=zipcodes_boundaries[zipcodes_boundaries.zipcode==zipcode].boundary.plot(ax=ax, color="red")
# # set_optimal_limits(ax, zipcodes_boundaries[zipcodes_boundaries.zipcode==zipcode])
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
# # ax=mismatches[mismatches.zipcode=="1301"].set_geometry("geometry_right").plot()


In [ ]:
# zipcodes_boundaries[zipcodes_boundaries.zipcode==zipcode]

In [ ]:
pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_zip_mismatches.pdf")

for zipcode in tqdm(zip_mismatches.postcode.value_counts().index):
    
    mism=zip_mismatches[zip_mismatches.postcode==zipcode].set_geometry("geometry_left")
    nis = ";".join(mism.municipality_id.unique()) 
    ax=mism.plot("zipcode", figsize=(10,10),alpha=0.8, legend=True)
    plt.title(f"{zipcode} ({nis})")
    # ax=region[region.postcode==zipcode].plot(figsize=(15,15),alpha=0.8, color="green")
    # ax=mismatches[mismatches.postcode==zipcode].set_geometry("geometry_left").plot(ax=ax, color="red")
    zip_bnd = zipcodes_boundaries[zipcodes_boundaries.zipcode==zipcode]
    if zip_bnd.shape[0]>0:
        ax=zip_bnd.boundary.plot(ax=ax, color="red")
    set_optimal_limits(ax, pd.concat([zip_bnd[["geometry"]], mism[["geometry_left"]].rename(columns={"geometry_left": "geometry"})]))
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    pdf.savefig(bbox_inches='tight')#, dpi=75)
    plt.close()
    
pdf.close()

# Conflict with NIS code

In [ ]:
download_if_nexist("https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_31370_20200101.shp.zip",
                   "data/stat_sectors_2020.zip")
statistical_sectors = gpd.read_file("zip://data/stat_sectors_2020.zip/sh_statbel_statistical_sectors_20200101.shp")
statistical_sectors["CNIS5_2020"] = statistical_sectors["CNIS5_2020"].astype(str)

In [ ]:
# Group (with "dissolve") sectors per NIS code
nis_boundaries = statistical_sectors[["CNIS5_2020", "T_MUN_FR", "T_MUN_NL", "geometry"]].dissolve(by="CNIS5_2020").reset_index()
nis_boundaries = nis_boundaries.rename({"CNIS5_2020": "niscode"}, axis=1)
nis_boundaries = nis_boundaries.to_crs(crs)

In [ ]:
nis_boundaries["buffer"] = nis_boundaries.buffer(-50)

In [ ]:
region_niscode = gpd.sjoin(region, nis_boundaries.set_geometry("buffer"))

In [ ]:
# region_niscode[region_niscode.streetname=="Place Albert Ier (MT)"][["municipality_id", "niscode"]].drop_duplicates()

In [ ]:
nis_mismatches = region_niscode[region_niscode.municipality_id !=region_niscode.niscode]
nis_mismatches

In [ ]:
nis_mismatches.drop(["geometry_left", "geometry_right", "index_right"], axis=1).to_excel(f"{output_dir}/best_anomalies_{region_name}_nis_mismatches.xlsx")

In [ ]:
pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_nis_mismatches.pdf")

for niscode in tqdm(nis_mismatches.municipality_id.value_counts().index):
    
    mism=nis_mismatches[nis_mismatches.municipality_id==niscode].set_geometry("geometry_left")
    
    ax=mism.plot("niscode", figsize=(10,10),alpha=0.8, legend=True)
    plt.title(niscode)
    # ax=region[region.postcode==zipcode].plot(figsize=(15,15),alpha=0.8, color="green")
    # ax=mismatches[mismatches.postcode==zipcode].set_geometry("geometry_left").plot(ax=ax, color="red")
    nis_bnd = nis_boundaries[nis_boundaries.niscode==niscode]
    if nis_bnd.shape[0]>0:
        ax=nis_bnd.boundary.plot(ax=ax, color="red")
    set_optimal_limits(ax, pd.concat([nis_bnd[["geometry"]], mism[["geometry_left"]].rename(columns={"geometry_left": "geometry"})]))
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    pdf.savefig(bbox_inches='tight')#, dpi=75)
    plt.close()
    
pdf.close()

# Street names incoherence

In [ ]:
# region.groupby(["streetname", "postcode"]).dissolve()

In [ ]:
region_streets = region[["streetname", "postcode", "geometry", "municipality_id"]].dissolve(["streetname", "postcode"])
region_streets["buffer"] = region_streets.buffer(100)
region_streets = region_streets.reset_index()
region_streets

In [ ]:
connected_streets = gpd.sjoin(region_streets, region_streets.set_geometry("buffer"))
connected_streets = connected_streets[connected_streets.streetname_left < connected_streets.streetname_right]

In [ ]:
connected_streets

In [ ]:
connected_streets[connected_streets.streetname_right=="Avenue de Tervuren"]

In [ ]:
for side in ["right", "left"]:
    connected_streets[[f"split_{side}_1", f"split_{side}_2"]] = connected_streets[f"streetname_{side}"].str.extract("^(Avenue|Rue|Chaussée|Boulevard|Drève|Clos|Square)(.*)$", flags=re.IGNORECASE)

In [ ]:
def single_digit_diff(a, b):
    if pd.isnull(a) or pd.isnull(b) or len(a)!= len(b):
        return False
#     print(a, b)
    diffs = list(difflib.ndiff([a], [b]))
#     print(diffs)
    if len(diffs) != 4:
        return False
    
    diff_pos_a = diffs[1].count(" ")
    diff_pos_b = diffs[3].count(" ")

    return a[diff_pos_a-1].isdigit() and b[diff_pos_b-1].isdigit()

In [ ]:
connected_streets["jaro"] =connected_streets.progress_apply(lambda row: jellyfish.jaro_winkler_similarity(row["streetname_left"],
                                                                                                          row["streetname_right"]), axis=1)

connected_streets["levenshtein"] =connected_streets.progress_apply(lambda row: jellyfish.damerau_levenshtein_distance(row["streetname_left"],
                                                                                                          row["streetname_right"]), axis=1)

connected_streets["jaro_split"] =connected_streets.progress_apply(lambda row: null_jaro(row["split_left_2"],
                                                                                        row["split_right_2"]), axis=1)

# single_last_letter : True if the only difference is the last letter, and the penultimate letter is a space
connected_streets["single_last_letter"] = (connected_streets.streetname_left.str.len() == connected_streets.streetname_right.str.len()) & \
                                            (connected_streets.streetname_left.str[:-2] == connected_streets.streetname_right.str[:-2]) & \
                                            (connected_streets.streetname_left.str[-2] == " ")

In [ ]:
connected_streets["single_digit_diff"] = connected_streets.progress_apply(lambda row: single_digit_diff(row["streetname_left"],
                                                                                        row["streetname_right"]), axis=1)

In [ ]:
connected_streets[connected_streets.single_digit_diff]

In [ ]:
streetname_mismatches = connected_streets[((connected_streets.levenshtein<=1) 
                                          | (connected_streets.jaro >=0.96)
                                          | ((connected_streets.jaro_split >= 0.96 ) & 
                                             (connected_streets.split_left_1.str.lower() == connected_streets.split_right_1.str.lower()))) &
                                         ~connected_streets.single_last_letter &
                                         ~connected_streets.single_digit_diff
                                         ]
                                         
streetname_mismatches

In [ ]:
# streetname_mismatches.sort_values("jaro")

In [ ]:
for (case, str_msmtch) in [("same_nis", streetname_mismatches[streetname_mismatches.municipality_id_left == streetname_mismatches.municipality_id_right]),
                           ("diff_nis", streetname_mismatches[streetname_mismatches.municipality_id_left != streetname_mismatches.municipality_id_right]),
                          ]:
    
    if str_msmtch.shape[0]==0:
        print("No case for", case)
        continue
    pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_close_streetnames_{case}.pdf")

    for i, rec in tqdm(str_msmtch.sort_values(["levenshtein", "jaro"], ascending=[True, False]).iterrows(),
                      total=str_msmtch.shape[0]):
    #     print(rec)
    #     rec = streetname_mismatches.sort_values("levenshtein").iloc[[i]]
        r1 = rec[["streetname_left", "postcode_left",  "municipality_id_left", "geometry_left"]].rename({"streetname_left":"streetname", 
                                                                                                         "postcode_left": "postcode", 
                                                                                                         "municipality_id_left": "municipality_id", 
                                                                                                         "geometry_left":"geometry"})
        r2 = rec[["streetname_right", "postcode_right", "municipality_id_right", "geometry_right"]].rename({"streetname_right":"streetname", 
                                                                                                            "postcode_right": "postcode", 
                                                                                                            "municipality_id_right": "municipality_id", 
                                                                                                            "geometry_right":"geometry"})
        
            
            
        r = gpd.GeoDataFrame([r1, r2])

        r["name"] = r["streetname"]+", "+r["postcode"]+"/"+r["municipality_id"]
        ax=r.plot("name", legend=True)
        set_optimal_limits(ax, r)
        ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

        pdf.savefig(bbox_inches='tight')#, dpi=75)
        plt.close()

    pdf.close()
    

In [ ]:
# gpd.GeoDataFrame(region[(region.streetname.str.startswith("Avenue de Tervu")) & (region.postcode=="1150")]).explore("streetname")

# Metrics

In [ ]:
# get_max_delta_ratio(street_bloc)

## Sinuosity

In [ ]:
sin_par = []
for parity in [0, 1]:
    sin= region[(region.house_number_num.mod(2)==parity)].groupby(["streetname", "postcode"]).progress_apply(bloc_sinuosity)#.sort_values(na_pos="first")
    sin = sin.sort_values(na_position="first").rename("sinuosity").reset_index()
    sin["parity"]=parity
    display(sin)
    sin_par.append(sin)
sinuosity= pd.concat(sin_par)
sinuosity = sinuosity[(sinuosity.sinuosity.notnull()) & (sinuosity.sinuosity<10**10)].sort_values("sinuosity", ascending=False).reset_index(drop=True)
sinuosity

In [ ]:
# sinuosity = sinuosity[sinuosity.sinuosity<10**10]
# region

In [ ]:
sw_sin_par = []
for parity in [0, 1]:
    sin= region[(region.house_number_num.mod(2)==parity)].groupby(["streetname", "postcode"]).progress_apply(sliding_sinuosity)#.sort_values(na_pos="first")
    sin = sin.sort_values(na_position="first").rename("sw_sinuosity").reset_index()
    sin["parity"]=parity
    display(sin)
    sw_sin_par.append(sin)
sw_sinuosity= pd.concat(sw_sin_par)
sw_sinuosity = sw_sinuosity[(sw_sinuosity.sw_sinuosity.notnull()) & (sw_sinuosity.sw_sinuosity<10**10)].sort_values("sw_sinuosity", ascending=False)
sw_sinuosity = sw_sinuosity.reset_index(drop=True)
sw_sinuosity

## Length

In [ ]:
#Add parity
# length = region.groupby(["streetname", "postcode"]).progress_apply(bloc_length)#.sort_values(na_pos="first")
# length = length.sort_values(na_position="first").rename("length").reset_index()
# length

In [ ]:
region_lenghts = []
for parity in [0,1]:
    region_par = region[(region.house_number_num.mod(2)==parity)].copy()

    length = region_par.groupby(["streetname", "postcode"]).progress_apply(bloc_length)#.sort_values(na_pos="first")
    length = length.sort_values(na_position="first", ascending=False).rename("length").reset_index()
    
    region_lenghts.append(length.assign(parity=parity))
length = pd.concat(region_lenghts)
length = length.sort_values("length", ascending=False).reset_index(drop=True)
length

## Distance to previous

In [ ]:
region_pars = []
for parity in [0,1]:
    region_par = region[(region.house_number_num.mod(2)==parity)].copy()

    region_par["dist_to_prev"] = region_par.distance(region_par.shift(1))
    region_par["dist_to_prev2"] = region_par.distance(region_par.shift(2))
    region_par["is_new_bloc"] = (region_par[["streetname", "postcode"]] !=  region_par[["streetname", "postcode"]].shift(1)).any(axis=1)
    region_par.dist_to_prev = region_par.dist_to_prev.where(~region_par.is_new_bloc, pd.NA)
    
    region_par.dist_to_prev2 = region_par.dist_to_prev2.where(~region_par.is_new_bloc, pd.NA)
    region_par.dist_to_prev2 = region_par.dist_to_prev2.where(~region_par.is_new_bloc.shift(1).astype(bool), pd.NA)
    
    
    region_pars.append(region_par.assign(parity=parity))
region_pars = pd.concat(region_pars)

In [ ]:
idx_max= region_pars.groupby(["streetname", "postcode", "parity"]).dist_to_prev.idxmax()
dist_to_prev = region_pars.loc[idx_max.dropna().values].sort_values("dist_to_prev", ascending=False)
dist_to_prev = dist_to_prev[["streetname", "postcode", "parity", "dist_to_prev", "house_number", "house_number_num"]].reset_index(drop=True)
dist_to_prev

In [ ]:
dist_to_prev.dist_to_prev.describe(percentiles=[0.5, 0.90, .95, .99, .999])

## Delta dist to prev

In [ ]:
idx_max= region_pars.groupby(["streetname", "postcode", "parity"]).dist_to_prev.idxmax()
delta_dist_to_prev =  region_pars.loc[idx_max.dropna().values].rename(columns={"dist_to_prev":"max_dist_to_prev"})#.sort_values("dist_to_prev")[["streetname", "postcode", "parity", "dist_to_prev", "house_number"]]
delta_dist_to_prev

In [ ]:
delta_dist_to_prev = delta_dist_to_prev.merge(region_pars.groupby(["streetname", "postcode", "parity"]).dist_to_prev.median().rename("median_dist_to_prev").reset_index())
delta_dist_to_prev["median_dist_to_prev"] = delta_dist_to_prev["median_dist_to_prev"].where(delta_dist_to_prev.median_dist_to_prev> 5, 0)

delta_dist_to_prev["delta_dist_to_prev"] = delta_dist_to_prev.max_dist_to_prev / delta_dist_to_prev.median_dist_to_prev

delta_dist_to_prev = delta_dist_to_prev[~delta_dist_to_prev.delta_dist_to_prev.isnull() & (delta_dist_to_prev.delta_dist_to_prev< np.inf)].sort_values("delta_dist_to_prev", ascending=False)
delta_dist_to_prev = delta_dist_to_prev[["streetname", "postcode", "parity", "house_number", "delta_dist_to_prev" ]].reset_index(drop=True)
delta_dist_to_prev

In [ ]:
# delta_dist_to_prev[delta_dist_to_prev.delta_dist_to_prev>1000]

## prev_to_prev2_ratio

In [ ]:
# region_pars[region_pars.dist_to_prev>region_pars.dist_to_prev2+100]



In [ ]:
region_pars["prev_to_prev2_ratio"] = (region_pars.dist_to_prev/region_pars[["dist_to_prev2"]].assign(m=10).max(axis=1,  skipna=False))#.replace(np.inf, pd.NA)
region_pars["prev_to_prev2_ratio"]

In [ ]:
idx_max= region_pars.groupby(["streetname", "postcode", "parity"]).prev_to_prev2_ratio.idxmax()
prev_to_prev2_ratio = region_pars.loc[idx_max.dropna().values].sort_values("prev_to_prev2_ratio", ascending=False)
prev_to_prev2_ratio = prev_to_prev2_ratio[["streetname", "postcode", "parity", "prev_to_prev2_ratio", "house_number"]].reset_index(drop=True)
prev_to_prev2_ratio

In [ ]:
prev_to_prev2_ratio.iloc[0:60]

In [ ]:
region_pars.sort_values("prev_to_prev2_ratio", ascending=False)

## Delta ratio

In [ ]:
# To avoid to compare all pairs of addresses in all street, we only consider streets with a sinuosity above 1.5 
# (assuming that "straight streets" won't have high delta ratio)

region_sel = region.merge(sinuosity[sinuosity.sinuosity>1.5][["streetname", "postcode"]])
region_sel

In [ ]:
delta_par = []
for parity in [0, 1]:
    dlt_par= region_sel[(region_sel.house_number_num.mod(2)==parity)].groupby(["streetname", "postcode"]).progress_apply(get_max_delta_ratio)#.sort_values(na_pos="first")
    dlt_par = dlt_par.apply(pd.Series).rename(columns={0: "delta_ratio", 1: "house_number" })#.reset_index()
    dlt_par = dlt_par.sort_values("delta_ratio", na_position="first", ascending=False)#.reset_index()
    dlt_par["parity"]=parity
    display(dlt_par)
    delta_par.append(dlt_par)
delta_ratio= pd.concat(delta_par)

delta_ratio = delta_ratio.sort_values("delta_ratio", ascending=False).reset_index()
delta_ratio

In [ ]:
# delta_ratio

## Consolidate

In [ ]:
metrics = {
    "dist_to_prev" :      dist_to_prev,
    "delta_dist_to_prev": delta_dist_to_prev,
    "sinuosity":          sinuosity,
    "sw_sinuosity":       sw_sinuosity,
    "length":             length,
    "delta_ratio":        delta_ratio,
    "prev_to_prev2_ratio":   prev_to_prev2_ratio
}

In [ ]:
delta_ratio

In [ ]:
with open(f"{data_dir}/metrics_{region_name}.pkl", "wb") as pkl:
    pickle.dump(metrics, pkl, pickle.HIGHEST_PROTOCOL)

region.to_pickle(f"{data_dir}/data_{region_name}.pkl")

In [ ]:
# Start from here to plot without recomputing all metrics
with open(f"{data_dir}/metrics_{region_name}.pkl", "rb") as pkl:
    metrics = pickle.load(pkl)#, pickle.HIGHEST_PROTOCOL)
region = pd.read_pickle(f"{data_dir}/data_{region_name}.pkl")

In [ ]:
# region_name

In [ ]:
for m in metrics:
    metrics[m] = metrics[m].reset_index(drop=True).copy()
#     metrics[m]["f"{m}_topn""]= False
#     metrics[m].loc[metrics[m].shape[0]-topn::, f"{m}_topn"]=True
    metrics[m][f"{m}_ranking"] = metrics[m].index+1
    if "house_number" in metrics[m]:
        metrics[m] = metrics[m].rename(columns={"house_number": f"{m}_house_number"})
        
    
#     dist_to_prev

In [ ]:
from functools import reduce
glob_metrics = reduce((lambda x, y: x.merge(y, how="outer")), metrics.values())

In [ ]:
glob_metrics


In [ ]:
glob_metrics[glob_metrics.streetname=="Chaussée de Wavre"]

## Show

In [ ]:
osm_crs=  'epsg:4326'

In [ ]:
# region

In [ ]:
k = 0
metric_name = "dist_to_prev"
metric_name = "sinuosity"
metric_name = "delta_ratio"

metric = metrics[metric_name]
print(metric.iloc[k])
street_bloc = get_street_bloc(region, 
                              metric.iloc[k].streetname, 
                              metric.iloc[k].postcode,
                              metric.iloc[k].parity)
    
street_bloc

In [ ]:
street_bloc = get_street_bloc(region, 
                  "Bredabaan", #"Rue François Michoel", 
                  "2990", #"4845",
                  1)
street_bloc

In [ ]:
plot_street_bloc(street_bloc, "title")

In [ ]:
street_bloc

In [ ]:
plot_street_bloc_plotly(street_bloc)
None

In [ ]:
# street_bloc.explore()

## PDF

In [ ]:
# make_table(street_bloc)

In [ ]:
metric

In [ ]:
thresholds = {
    "dist_to_prev": 1000,
    "length":5000,
    "delta_ratio": 1.0,
    "delta_dist_to_prev":50,
    "sinuosity": 10,
    "sw_sinuosity": 10,
    "prev_to_prev2_ratio": 10
}

In [ ]:
metrics.keys()
# region

In [ ]:
for metric_name in metrics:
# for metric_name in ['prev_to_prev2_ratio']:
    metric=metrics[metric_name]
    print(metric_name)
    pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_{metric_name}.pdf")
    
    if metric_name in thresholds:
        metric = metric[metric[metric_name]>thresholds[metric_name]]
    for i, metr in tqdm(metric.iloc[0:topn].iterrows(), total=min(topn, metric.shape[0])):# in trange(1, min(51, metric.shape[0])):

        
        street_bloc = get_street_bloc(region, 
                                      metr.streetname, 
                                      metr.postcode,
                                      metr.parity)
        sb = street_bloc.iloc[0]
        title = f"{sb.streetname} - {sb.postcode}/{sb.municipality_id} - {sb.postname} - {sb.municipality} - parity: {metr['parity']} "
        title += f"\n{metric_name}: {metr[metric_name]:.2f}"
#         title = street_bloc.iloc[0]["streetname"]+ " - " + street_bloc.iloc[0]["postcode"] + \
#                 " - " + street_bloc.iloc[0]["postname"] + " - " + street_bloc.iloc[0]["municipality"]+" - parity: "+str(metr["parity"])+"\n"+\
#                 f"{metric_name}: {metr[metric_name]:.2f}"
        if f"{metric_name}_house_number" in metr:
            title += f" (hn: {metr[f'{metric_name}_house_number']})"

        make_table(street_bloc, title)
        pdf.savefig(bbox_inches='tight')
        plt.close()
        plot_street_bloc(street_bloc, title)
        pdf.savefig(bbox_inches='tight')
        plt.close()
    pdf.close()

In [ ]:
f"{output_dir}/best_anomalies_{region_name}_{metric_name}.pdf"

### Consolidated pdf

In [ ]:
# glob_metrics[(glob_metrics[[f"{m}_ranking" for m in metrics]]  < topn).fillna(False).any(axis=1)]


In [ ]:
glob_metrics_topn = glob_metrics[(glob_metrics[[f"{m}_ranking" for m in metrics]]  < topn).fillna(False).any(axis=1)]

glob_metrics_topn = glob_metrics_topn[pd.concat([glob_metrics_topn[m] > thresholds[m] for m in metrics], axis=1).any(axis=1)]


glob_metrics_topn = glob_metrics_topn.sort_values("dist_to_prev", ascending=False)
glob_metrics_topn

In [ ]:
glob_metrics_topn["global_ranking"] = glob_metrics_topn[glob_metrics_topn.columns[glob_metrics_topn.columns.str.endswith("_ranking")]].fillna(100).apply(np.log).sum(axis=1)

In [ ]:
glob_metrics_topn = glob_metrics_topn.sort_values("global_ranking")
glob_metrics_topn

In [ ]:
pdf = PdfPages(f"{output_dir}/best_anomalies_{region_name}_consolidated.pdf")

for i, metr in tqdm(glob_metrics_topn.iterrows(), total=glob_metrics_topn.shape[0]):# in trange(1, min(51, metric.shape[0])):
    street_bloc = get_street_bloc(region, 
                                  metr.streetname, 
                                  metr.postcode,
                                  metr.parity)
    
    sb = street_bloc.iloc[0]
    title = f"{sb.streetname} - {sb.postcode}/{sb.municipality_id} - {sb.postname} - {sb.municipality} - parity: {metr['parity']} "

    make_table(street_bloc, title)
    pdf.savefig(bbox_inches='tight')
    plt.close()

    fig, ax = plt.subplots(nrows=2, figsize=(10,10), gridspec_kw={'height_ratios':[0.85, 0.15]})
    plot_street_bloc(street_bloc, title, ax=ax[0])
    make_metric_table(metr, metrics, ax=ax[1])

    pdf.savefig(bbox_inches='tight')
    plt.close()
pdf.close()

In [ ]:
region_name